In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


# Carregue seus dados
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# Defina suas variáveis dependentes e independentes
y_train = df_train["smoking"]
X_train = df_train.drop("smoking", axis=1)
X_test = df_test

# Divida seus dados em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=5)

# Crie um modelo K-Nearest Neighbors
knn = KNeighborsClassifier(n_jobs=-1)

# Defina os hiperparâmetros que você deseja otimizar
parameters = {
    'n_neighbors': [3, 5, 7, 9],  # Número de vizinhos a serem considerados
    'weights': ['uniform', 'distance'],  # Tipo de peso para os vizinhos
    'p': [1, 2],  # P-norm (1 para distância de Manhattan, 2 para distância Euclidiana)
}

# Realize uma pesquisa em grade para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(knn, parameters, n_jobs=-1, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=5))
grid_search.fit(X_train, y_train)

# Imprima o melhor modelo encontrado durante a pesquisa em grade
print("Melhor modelo:", grid_search.best_estimator_)

# Imprima a pontuação do melhor modelo
score_best = grid_search.best_score_
print("Pontuação do melhor modelo:", score_best)

# Faça previsões de probabilidade no conjunto de teste
y_test_probabilities = grid_search.predict_proba(X_test)  # Isso retorna as probabilidades em vez de classes 0 ou 1

# Crie um DataFrame de submissão com as probabilidades de previsão
df_submission = pd.DataFrame({"id": df_test["id"], "smoking": y_test_probabilities[:, 1]})  # Use [:, 1] para obter a probabilidade da classe positiva

# Salve o DataFrame de submissão em um arquivo CSV
df_submission.to_csv("sample_submission_knn_probabilities.csv", index=False)


KeyboardInterrupt: 